# Data analysis for converting comb tooth beats vs. time, to frequency vs. time, to transmission versus frequency.

In [ ]:
%matplotlib inline
import numpy as np
from numpy import average, hstack, argsort, ones, sort, diff, where

from itertools import cycle
import matplotlib.pyplot as plt

from scipy.io import savemat, loadmat
from matplotlib import pylab as pl
import matplotlib.pyplot as plt

from scipy.ndimage.measurements import label, find_objects, center_of_mass
from scipy.ndimage.filters import sobel, gaussian_filter1d, median_filter, laplace, uniform_filter1d
from scipy.signal import find_peaks_cwt


from scipy.interpolate import InterpolatedUnivariateSpline, interp1d, UnivariateSpline, splrep

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot,iplot_mpl
init_notebook_mode()

In [ ]:
def rollingWindow(a, window, edge = 'copy'):
    if edge == 'copy':
        extended = np.zeros(a.shape[:-1] + (a.shape[-1]+window-1, ))
        extended[..., int(window/2.-1): -int(window/2.)] = a[...]
        extended[..., :int(window/2.+1)]= a[...,0]
        extended[..., -int(window/2.):]= a[...,-1]
        a = extended
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def rollingAverage(a,win_size=10):
    out=average(rollingWindow(a, win_size), -1)
    #out1=np.append(np.zeros(int(window/2)),out)
    #out2=out1[:-int(window/2)] #these two modiciations make the filter symetric
    return out

Below, we select the file we wish to process, and a sample of the Transmission data, and the demodulation channel is shown. If you see no resonances in the transmisison spectrum, or no peaks in the demodulation channel, something has gone wrong in data collection and processing is impossible.

In [ ]:
filename='rb_1555_1570_200nm_1'
#filename='test'
d = loadmat(filename + '.mat', squeeze_me = True)
scale = 1e-9
t_range = [int(1e3),int(1e7)]
ch_n = 2

ch = [ d['ch{:}'.format(c+1)][:,t_range[0]:t_range[-1]] for c in range(ch_n) ]
trans = d['trans'][:,t_range[0]:t_range[-1]]

# Start from 0
trans[0] = (trans[0]-trans[0][0])*scale
for i in range(len(ch)):
    ch[i][0] = (ch[i][0]-ch[i][0][0])*scale
ch_nu = pl.arange(0,len(ch))

f, ax = pl.subplots(2,1, figsize=(10,4))
ax[0].plot(trans[0][::50], trans[1][::50])

for c in ch:
    ax[1].plot(c[0][::500],c[1][::500])
#ax[1].set_xlim(5,5+.1)
iplot_mpl(f)

Filter the demodulation signal to enhance the peak. We use `ch_filter` function and a `th` threshold value. The figure shows a fragment of the demodulation signal, filtered and with the threshold.

It is typically inadvisable to tune the `dem_alpha` (the filter corner) but tuning of `dem_th` is required. You want it sufficiently above the noise floor to reject any false comb teeth, but low enough to still see each tooth. 

To check sucess, see how many peaks the program had to manually insert, the number should be approximately less than 5000


In [ ]:
dem_th =6e-7
dem_alpha = 90

## Demod signal
def ch_filter(d, alpha = 100):
    d5 = gaussian_filter1d(d,3)
    d6 = laplace(d5)
    return(d5-alpha*d6)

avgsize=4
f, ax = pl.subplots(len(ch),1, figsize=(20,8))
for i, c in enumerate(ch):
    #ax[i].plot(c[0],c[1],'k',lw=2)
    ax[i].plot(c[0],ch_filter(c[1],dem_alpha), '-',lw=2,)
    ax[i].set_xlim(10.2,10.23)
    ax[i].set_ylim(-1e-8,+15e-7)
    ax[i].axhline(dem_th,ls='--',c = 'k')




Below, we analyze the pattern of peaks, and force pattern completion to account for any mis-fit peaks. The output graph, shows the pattern around one of the breaks in the expected pattern, and can be used to diagnosis whether the threshold is too low (noise is inadvertently identified as a peak) or to high (a peak is inadvertently ignored).

`new points` is the number of breaks in the expected pattern, and you should tune `dem_th` until the value is less than about 5000 (but this is just an order of magnitude, dont worry if its a little more).

In [ ]:
def get_peaks(t, c, min_dist = 0.000000001):
    ## isolate each peak
    l, n = label(c)
    #peaks = r_[[average(t[l==i]) for i in  arange(1,n)]].T
    peaks = np.array([average(t[obj]) for obj in find_objects(l)])
    size = np.array([len(l[obj])*(t[1]-t[0]) for obj in find_objects(l)])
    peaks = peaks[:-1][diff(peaks)>min_dist]
    return peaks, size


peaks = [get_peaks(c[0],ch_filter(c[1],dem_alpha)>dem_th) for c in ch]

# # deletes repeated peak lcoations, if coincidently present
# repeated1=[]
# repeated2=[]
# for i in range(len(val)):
#     repeated1.append(np.where(peaks[:][0][0]==val[i])[0][0])
#     repeated2.append(np.where(peaks[:][1][0]==val[i])[0][0])


peaks_c = hstack([ones(len(peaks[i][0])) * ch_nu[i] for i in range(len(peaks)) ]) # Channel
peaks_t = hstack([peaks[i][0] for i in range(len(peaks)) ]) # T

o_peaks_c = peaks_c[argsort(peaks_t)].astype(np.int16)
o_peaks_t = sort(peaks_t)
    
val=np.where(np.diff(o_peaks_t)==0)[0]
o_peaks_t=np.delete(o_peaks_t,val)
o_peaks_c=np.delete(o_peaks_c,val)

print('Found {:} peaks'.format(len(o_peaks_c)))

patch_pattern = cycle([1,0,0,1])
do_patch = True

def patch(t,c, pattern):
    new_p = []
    rem_p = []
    for i in range(len(c)-1):
        step = 0
        symbol = []
        while True:
            x = next(pattern)
            if (i+1) >= len(c): break
            if c[i+1] != x:  #look ahead
                symbol.append(x)
                if len(symbol)>4:  #it probably means we need to remove the peak
                    rem_p.append(i)
                    for j in range(3):
                        next(pattern)
                    break
            else: ## add all the missing patterns
                for j,x in enumerate(symbol):
                    val=(t[i] + (t[i+1]-t[i])*(j+1)/(len(symbol)+1), x)
                    new_p.append(val)
                break
    
    #t = np.delete(t,rem_p)
    #c = np.delete(c,rem_p)
    
    new_p = np.array(new_p).T
    return new_p

# We expect peaks beeing 12344321.. so let's make sure this sequence repeats
t,c = o_peaks_t[:], o_peaks_c[:]
f,axes = pl.subplots(3,1, figsize=(20,6), sharex=True)

# Before patch
axes[0].plot(t[1:],diff(c),'o-')
axes[0].plot(t,c,'o-')
print('Original number of points {:}'.format(len(t)))

new_p = patch(t,c,patch_pattern)        
if do_patch:
    t = np.r_[t,new_p[0]]
    c = np.r_[c,new_p[1]]
    c = c[argsort(t)]
    t = sort(t)

#removes repeated indices
#t=np.array(sorted(list(set(t))))

#After patch
axes[1].plot(t[1:],diff(c),'o-')
axes[1].plot(t,c,'o-')
print('New points {:}'.format(len(new_p[0])))

axes[2].plot(ch[0][0],ch[0][1])
axes[2].plot(ch[1][0],ch[1][1])

#axes.set_ylim(0,4)
t0 = new_p[0,100]
axes[0].set_xlim(t0- 2e-2, t0+2e-2)
axes[2].set_ylim(0,5e-7)

Now, based on the corrected pattern of modulation peaks, we can extract frequency versus time. This is done by using the comb spacing as a ruler, counting the frequency we have sweeped in terms of the number of comb teeth we hve seen, then interpolating between teeth to fill out the rest of the spectrum.

This gives us an estimate for the frequency we have swept versus the time of the sweep, which combined with the initial wavelength, allows frequency knowledge of the laser at the Mhz level for the entire sweep.

In [ ]:

# Unwrap the channels
mod = [25,100]       # Demod freq
mod = [10,40]       # Demod freq
rep_rate = 250 # Menlo reprate

deri = diff(c)

deri2 = np.r_[0,diff(deri)]
n_p = (c[1:]+1)

n_p[c[1:]==0] = mod[0]
n_p[c[1:]==1] = mod[1]

###############
## First unwrap the two halves
n_p = where(deri > 0, n_p, rep_rate-n_p) # The zero point
n_p = where(np.logical_and(deri == 0, deri2>0), mod[0], n_p )

## Every passage from zero, add rep rate
n_p = n_p + np.cumsum(where(np.logical_and(deri == 0, deri2>0), 1, 0) )*rep_rate

## Trendline
freq_fit = np.poly1d(np.polyfit(t[1::100], n_p[::100],2))

#find points that are clearly off and remove them
#rem_p = where(diff(freq_f(t)-freq_fit(t))>205)[0]
#print(rem_p[:10])
n_t = t[1:]


## T vs Frequency !!
#freq_f = lambda x: np.interp(x, t[1:], n_p)
#feq_f = InterpolatedUnivariateSpline(n_t, n_p)
#Piece_wise spline
block = 50
np_spline = []
i = 0
while i< len(n_p):
    sl = slice(i,min(i+block,len(n_p)))
    np_spline.append(UnivariateSpline(n_t[sl], n_p[sl], k=5,s=50000)(n_t[sl]))
    i = i+block
    
n_p2 = np.hstack(np_spline)
freq_f = InterpolatedUnivariateSpline(n_t, n_p2)


Now, we plot the frequency versus time of the scan, with a linear trend removed. This is the deviation from a perfectly linear laser sweep, in Mhz.

Typical behavior depends on laser used, but fluctuation on a short time sclae (consitent with piezo motion) and slower drift (consitent with thermal drift) are common.

adjust the "cut" value below, to remove any spurios data at the end of the sweep.

In [ ]:
f,ax = pl.subplots(1,1, figsize=(8,4))
cut=int(65e5)
t0cut=trans[0,0:cut]
t1cut=trans[1,0:cut]

ax.plot(n_t,n_p2-freq_fit(n_t))
ax.plot(t0cut[::100],freq_f(t0cut[::100])-freq_fit(t0cut[::100]))
#ax.set_xlim(0,95)
ax.set_ylim(-2000,1500)
iplot_mpl(f)

#use this to make sure laser looks ok, and cut the the end of the scan if the sweep ends

Now we save the spectrum. This final result, is the obsevred transmission, versus frequency. Note, that the program has no way of telling which way you sweep, the units here are absolute frequency difference (in Mhz) from the start of the scan.

In [ ]:
## Save transmission and frequency in one file
savemat(filename+'_tr', {'data': np.vstack([freq_f(trans[0,0:cut]), trans[1,0:cut]],)}),
                        #'options': dict(t_range = t_range,
                        #                dem_th = dem_th,
                        #                dem_alpha = dem_alpha,
                        #                do_patch = do_patch,
                        #                patch_pattern = patch_pattern)})

In [ ]:
cut=int(65e5)
